# CUNY DATA 612 Project 1 | Global Baseline Predictors and RMSE
Ananya Punyatoya
06/07/2020

# Intoduction 
1. I have taken a relatively smaller and dense data set unlike real world project.
2. Each rows describes a particular user and each colums describes a particular book; Each cell represents a rating provided by an user for a particular book 
3. User bias and book bias are calculated for each user and each movie 
4. predicted rating = raw_avg(training data set only) + user_bias + book_bias
5. Used RMSE(root mean square error) to find the efficiencies of the model 

## importing Required library for building recomendation system 

In [1]:
import pandas as pd
import numpy as np


## Reading data from an excel file 

In [2]:
book_data = pd.read_excel(r"book_data.xlsx", encoding = 'unicode_escape',index_col=0 )
book_data = book_data.replace('?',np.nan)
book_data = book_data.values
book_data

array([[  5.,  nan,   4.,  nan,   4.],
       [  4.,   3.,   5.,   3.,   4.],
       [  4.,   2.,  nan,  nan,   3.],
       [  2.,   2.,   3.,   1.,   2.],
       [  4.,  nan,   5.,   4.,   5.],
       [  4.,   2.,   5.,   4.,   4.]])

## Spliting data into test and train set 

In [3]:
#creating index for tran and test data
pct_test = 0.2
n_test = int(round(pct_test * np.count_nonzero(~np.isnan(book_data)), 0))
non_na_index = np.argwhere(~np.isnan(book_data))
seq = np.arange(len(non_na_index[:, 0]))
test_index = np.random.choice(seq, n_test, replace = False)
train_index= np.delete(seq, test_index)


In [4]:
#split  data set  into test and train
train = np.copy(book_data)
train[non_na_index[test_index, 0], non_na_index[test_index, 1]] = np.nan
test = np.copy(book_data)
test[non_na_index[train_index, 0], non_na_index[train_index, 1]] = np.nan


In [5]:
train

array([[  5.,  nan,   4.,  nan,   4.],
       [ nan,  nan,   5.,  nan,   4.],
       [  4.,   2.,  nan,  nan,   3.],
       [ nan,   2.,   3.,   1.,   2.],
       [  4.,  nan,   5.,  nan,   5.],
       [  4.,   2.,   5.,   4.,   4.]])

## converting numpy array to pandas  data Frame 

In [6]:
train = pd.DataFrame(train,index=['p1','p2','p3','p4','p5','p6'],columns=['B1','B2','B3','B4','B5'])
train

,B1,B2,B3,B4,B5
p1,5.0,NaN,4.0,NaN,4.0
p2,NaN,NaN,5.0,NaN,4.0
p3,4.0,2.0,NaN,NaN,3.0
p4,NaN,2.0,3.0,1.0,2.0
p5,4.0,NaN,5.0,NaN,5.0
p6,4.0,2.0,5.0,4.0,4.0


In [7]:
test

array([[ nan,  nan,  nan,  nan,  nan],
       [  4.,   3.,  nan,   3.,  nan],
       [ nan,  nan,  nan,  nan,  nan],
       [  2.,  nan,  nan,  nan,  nan],
       [ nan,  nan,  nan,   4.,  nan],
       [ nan,  nan,  nan,  nan,  nan]])

In [8]:
test = pd.DataFrame(test,index=['p1','p2','p3','p4','p5','p6'],columns=['B1','B2','B3','B4','B5'])
test

,B1,B2,B3,B4,B5
p1,NaN,NaN,NaN,NaN,NaN
p2,4.0,3.0,NaN,3.0,NaN
p3,NaN,NaN,NaN,NaN,NaN
p4,2.0,NaN,NaN,NaN,NaN
p5,NaN,NaN,NaN,4.0,NaN
p6,NaN,NaN,NaN,NaN,NaN


# Building Raw average model 

In [9]:
# function to calculate mean of data frame 

def menaOfDf(df):
    count = df.count().sum()
    sum_total = df.sum().sum()
    mean = (sum_total/count)
    return mean


In [10]:
train_mean = menaOfDf(train)
train_mean

3.6000000000000001

In [11]:
#Creating a similar data frame with the existing one and fill with raw average value

def createAndFillAvg(df,val):
    df_predict= pd.DataFrame().reindex_like(df)
    df_predict= df_predict.fillna(val)
    return df_predict

train_predict = createAndFillAvg(train,train_mean)
test_predict  = createAndFillAvg(test,train_mean)

In [12]:
train_predict

,B1,B2,B3,B4,B5
p1,3.6,3.6,3.6,3.6,3.6
p2,3.6,3.6,3.6,3.6,3.6
p3,3.6,3.6,3.6,3.6,3.6
p4,3.6,3.6,3.6,3.6,3.6
p5,3.6,3.6,3.6,3.6,3.6
p6,3.6,3.6,3.6,3.6,3.6


## Evaluating Raw average model

In [13]:
# custom function to calculate rsme of 2 data frame
def rmse(df1,df2):
    se = ((df1-df2)**2).sum()
    se = se.sum()
    count = df1.count().sum()
    mse = (se/count)
    rmse = (mse)**0.5
    return rmse


In [14]:
rmse_train = rmse(train,train_predict)
rmse_test =  rmse(test,train_predict)

In [15]:
rmse_train

1.2

In [16]:
rmse_test

0.84852813742385713

## Rmse result of train and test data for Raw average model  
rmse_train = 1.2
rmse_test = 0.84852813742385713



# Building Baseline predictor model
## calculting book_bias

In [17]:
book_mean = train.mean()
book_mean

B1    4.250000
B2    2.000000
B3    4.400000
B4    2.500000
B5    3.666667
dtype: float64

In [18]:
book_bias = book_mean - train_mean
book_bias

B1    0.650000
B2   -1.600000
B3    0.800000
B4   -1.100000
B5    0.066667
dtype: float64

## calculating User_bias

In [19]:
user_mean = train.mean(axis=1)
user_mean

p1    4.333333
p2    4.500000
p3    3.000000
p4    2.000000
p5    4.666667
p6    3.800000
dtype: float64

In [20]:
user_bias = user_mean - train_mean
user_bias

p1    0.733333
p2    0.900000
p3   -0.600000
p4   -1.600000
p5    1.066667
p6    0.200000
dtype: float64

In [21]:
train_predict

,B1,B2,B3,B4,B5
p1,3.6,3.6,3.6,3.6,3.6
p2,3.6,3.6,3.6,3.6,3.6
p3,3.6,3.6,3.6,3.6,3.6
p4,3.6,3.6,3.6,3.6,3.6
p5,3.6,3.6,3.6,3.6,3.6
p6,3.6,3.6,3.6,3.6,3.6


## Adding book_bais and user_bais  to predict rating 

In [22]:
train_predict= train_predict.add(book_bias).add(user_bias,axis=0)
train_predict

,B1,B2,B3,B4,B5
p1,4.983333,2.733333,5.133333,3.233333,4.400000
p2,5.150000,2.900000,5.300000,3.400000,4.566667
p3,3.650000,1.400000,3.800000,1.900000,3.066667
p4,2.650000,0.400000,2.800000,0.900000,2.066667
p5,5.316667,3.066667,5.466667,3.566667,4.733333
p6,4.450000,2.200000,4.600000,2.700000,3.866667


## Rounding of any  rating >5  to 5 and  less than 1 to 1 

In [23]:
def roundingNumber(df):
    df[df>5] =5
    df[df<1]=1
    return df


In [24]:
train_predict = roundingNumber(train_predict)
train_predict

,B1,B2,B3,B4,B5
p1,4.983333,2.733333,5.0,3.233333,4.400000
p2,5.000000,2.900000,5.0,3.400000,4.566667
p3,3.650000,1.400000,3.8,1.900000,3.066667
p4,2.650000,1.000000,2.8,1.000000,2.066667
p5,5.000000,3.066667,5.0,3.566667,4.733333
p6,4.450000,2.200000,4.6,2.700000,3.866667


## Evaluation of Baseline predictor model  for training data

In [25]:
rmse_base_line_train = rmse(train,train_predict)
rmse_base_line_train

0.55651445024665203

## Evaluation of Baseline predictor model for test data

In [26]:
rmse_base_line_test = rmse(test,train_predict)
rmse_base_line_test

0.59670390945221352

## Performace improvement  

In [27]:
def improveInPerformance(df1, df2):
    return ((df1 - df2)/df1) * 100


In [28]:
improvement_train = improveInPerformance(rmse_train, rmse_base_line_train)
improvement_train

53.623795812778994

In [29]:
improvement_test = improveInPerformance(rmse_test, rmse_base_line_test)
improvement_test 

29.677769877636038


# Conclusion 

###   Baseline predictor shows much better results than the simple raw average model. Moreover, it  is behaving better in train data than test data sets. However, to improve this model, we can use collaborative filtering  (item based or user based) with a large volume of data set. As we can clearly see that Base line predictor is more reliable than Raw average , but result may vary with diffrent data set and high volume data. A larger dataset  and sophisticated model is  recommended to perform better  computation.
    

